In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

class AgentState(TypedDict):
    user_input: Annotated[str,lambda old,new:new]
    email_template: Annotated[str, lambda old, new: new]
    user_feedback: Annotated[str,lambda old,new:new]
    approved: Annotated[bool,lambda old,new:new]

In [2]:
from  langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3-groq-tool-use")
graph_builder = StateGraph(AgentState)

def draft_content(state:AgentState):
    if(state['user_feedback']):
        prompt = f"""
        Here is the current email template:
        ---
        {state['email_template']}
        ---
        Update it based on this feedback:
        "{state['user_feedback']}"
        Remeber-
        1. Only write the message and no need to write subject.
        2. Dont write any thing else like updating or here is your email.
        3. Dont not add any of yours comments
        """
    else:
         prompt = f"""
        Write a professional email based on this input:
        "{state['user_input']}"

        Use placeholders like {{name}}, {{email}}, {{company_name}} where needed.
        Remeber-
        1. Only write the message and no need to write subject.
        2. Dont write any thing else like updating or here is your email.
        3. Dont not add any of yours comments
        """
    response = llm.invoke(prompt)
    return {"email_template": response.content}

In [ ]:
def get_feedback(state:AgentState):
    print("\n📨 Draft Email:\n")
    print(state["email_template"])
    print("\n---")
    user_input = input("Approve this email? (yes or give feedback): ")
    if user_input.lower() in ["yes", "approve", "approved", "y"]:
        return {"approved": True}
    else:
        return {
            "user_feedback": user_input,
            "approved": False
        }

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from mailjet_rest import Client

In [5]:
import base64

def get_attachment(path):
    with open(path, "rb") as file:
        file_data = file.read()
        encoded = base64.b64encode(file_data).decode()
        return {
            "ContentType": "application/pdf",  # Change this based on file type
            "Filename": path.split("/")[-1],
            "Base64Content": encoded
        }

In [ ]:
def send_email_via_mailjet(to_email, to_name, subject, content, from_email="dragnoid121@gmail.com", from_name="AAK",attachment=None):
    api_key = os.getenv("MJ_APIKEY_PUBLIC")
    secret_key = os.getenv("MJ_APIKEY_PRIVATE")
    mailjet = Client(auth=("cca56ed08f5272f813370d7fc5a34a24", "60fb43675233e2ac775f1c6cb8fe455c"), version='v3.1')
    message_data = {
        "From": {"Email": from_email, "Name": from_name},
        "To": [{"Email": to_email, "Name": to_name}],
        "Subject": "hi",
        "TextPart": content,
        "HTMLPart": f"<pre>{content}</pre>"
    }

    if attachment:
        message_data["Attachments"] = [attachment]

    data = {"Messages": [message_data]}
    try:
        result = mailjet.send.create(data=data)
        print("Sent:", result.status_code, result.json())
        return result
    except Exception as e:
        print("❌ Error sending email:", e)
    


In [ ]:
import pandas as pd 
def send_emails(state: AgentState):
    user_confirmation = input("Are u sure you want to send the emails: Y or N")
    if(user_confirmation == "N"):
        return 
    else:
        template = state["email_template"]
        # attachment = get_attachment("brochure.pdf")  # Or docx, image, etc.

        df = pd.read_excel("C:/Users/Asus/AutoIntern/email.xlsx")
        for i, row in df.iterrows():
            content = template.format(
                name=row["name"],
                email=row["email"],
                company_name=row["company_name"]
            )
            result = send_email_via_mailjet(
                to_email=row["email"],
                to_name=row["name"],
                subject="Welcome to the Team!",
                content=content
                # attachment=attachment
            )
            df.at[i, 'Status'] = result.status_code
        df.to_excel("email_status.xlsx",index=False)
        
        return {}


In [8]:
def route_tools(state:AgentState):
    if(state['approved']):
        return "send_emails"
    else:
        return "llm_writer"

In [9]:
from langgraph.graph import StateGraph,START,END

graph_builder = StateGraph(AgentState)
graph_builder.add_node("llm_writer",draft_content)
graph_builder.add_node("get_feedback",get_feedback)
graph_builder.add_node("send_emails",send_emails)

graph_builder.add_edge(START,"llm_writer")
graph_builder.add_edge("llm_writer","get_feedback")
graph_builder.add_conditional_edges("get_feedback",route_tools)
graph_builder.add_edge("send_emails",END)

graph = graph_builder.compile()

In [10]:
def run_agent(user_input: str):
    initial_state = {
        "user_input": user_input,
        "email_template": "",
        "user_feedback": "",
        "approved": False
    }

    final_state = graph.invoke(initial_state)
    print("\n✅ Agent Finished.")

In [ ]:
run_agent("I want to send a welcome email to all the interns joining next week,emails are mentioned in the excel.")


📨 Draft Email:

Here is the professional email:

"Dear {name},

I am thrilled to welcome you to {company_name} as an intern next week! We are excited to have you join our team and contribute your skills.

Please find attached all the necessary information and documents you need to get started. If you have any questions or need further assistance, feel free to reach out to us at {email}.

Looking forward to seeing you soon!

Best regards,"

---

📨 Draft Email:

Here's a professional welcome email:

Dear {name},

On behalf of {company_name}, I am thrilled to welcome you to our team! We are excited to have you join us next week.

Please find attached a detailed schedule and orientation plan for your first week. If you have any questions or need assistance, feel free to reach out to us at {email}.

Looking forward to seeing you soon!

Best regards,
[Your Name]

---
Sent: 200 {'Messages': [{'Status': 'success', 'CustomID': '', 'To': [{'Email': 'alokkale121@gmail.com', 'MessageUUID': '5d36f